In [1]:
import pandas as pd
from IPython.display import display
import math
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

2023-03-16 22:06:49.929600: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 22:06:50.513429: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-16 22:06:50.513484: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-16 22:06:50.513489: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
dataset = pd.read_csv("../data/assignment_preprocessed.csv")
dataset.drop(['id','price_per_sqm', 'agent_id'], axis=1, inplace=True)

**Categorical to numerical data**
Can be done either with the use of mapping or with one-hot encoding. We will use mapping for the correlation analysis and one-hot encoding for the model training.

In [3]:
ad_type_mapping = {'simple': 1, 'up': 2, 'premium': 3, 'star': 4}
geo_mapping = {'northern sub': 1, 'south beach': 2, 'gentrification area': 3, 'beesy neighborhood': 4}
floor_mapping = {'ground-floor': 0, 'basement': -1, 'mezzanine': 0.5, 'semi-basement': -0.5}
subtype_mapping ={'apartment': 1, 'detached': 2, 'maisonette': 3, 'building': 4, 'villa': 5, 'studio': 6, 'other residential': 7, 'loft': 8, 'apartment complex': 9, 'bungalow': 10}

In [4]:
dataset['geography_name'] = dataset['geography_name'].map(geo_mapping).fillna(dataset['geography_name']).astype(int)
dataset['ad_type'] = dataset['ad_type'].map(ad_type_mapping).fillna(dataset['ad_type']).astype(int)
dataset['floor'] = dataset['floor'].map(floor_mapping).fillna(dataset['floor']).astype(float)
dataset['subtype'] = dataset['subtype'].map(subtype_mapping).fillna(dataset['subtype']).astype(int)

## Ways to identify the most important attributes in predicting the price of a property

### Correlation Analysis

In [5]:
def correlation_analysis(dataset, target_variable):
    # calculate correlation matrix
    corr_matrix = dataset.corr()

    # get correlation with target variable
    corr_with_target = corr_matrix[target_variable]

    # sort correlation values in descending order
    sorted_corr = corr_with_target.abs().sort_values(ascending=False)

    # print top 10 attributes with highest correlation
    return list(sorted_corr.iloc[1:11].keys())

In [6]:
correlation_analysis_list = correlation_analysis(dataset, 'price')

### Feature Importance using Decision Trees 

In [7]:
X = dataset.drop(['price'], axis=1)
y = dataset['price']

In [8]:
# create decision tree model
model = DecisionTreeRegressor()

# fit model to data
model.fit(X, y)

DecisionTreeRegressor()

In [9]:
# get feature importances
importances = model.feature_importances_

# sort importances in descending order
sorted_importances = importances.argsort()[::-1]

# print top 10 attributes with highest importance
decision_tree_list = list(X.columns[sorted_importances[:10]])

### Feature Importance using Random Forest

In [10]:
rf = RandomForestRegressor(n_estimators=150)
rf.fit(X, y)

RandomForestRegressor(n_estimators=150)

In [11]:
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': rf.feature_importances_})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

In [12]:
random_forest_list = list(feature_importance.head(10)['Feature'])

## Price Prediction

In [13]:
# attributes that at least two of the ways found as an important feature
important_attributes = list(set(correlation_analysis_list).intersection(decision_tree_list, random_forest_list) | set(decision_tree_list).intersection(random_forest_list) | set(correlation_analysis_list).intersection(random_forest_list))

In [14]:
dataset = pd.read_csv("../data/assignment_preprocessed.csv")
dataset.drop(['id','price_per_sqm', 'agent_id'], axis=1, inplace=True)

In [15]:
def oneHotEncode(df,colNames):
    for col in colNames:
        if( df[col].dtype == np.dtype('object')):
            dummies = pd.get_dummies(df[col],prefix=col)
            df = pd.concat([df,dummies],axis=1)

            #drop the encoded column
            df.drop([col],axis = 1 , inplace=True)
    return df

In [16]:

def get_results(y_test, y_pred): # Evaluate model performance
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print('Mean squared error:', round(mse))
    print('Root MSE:', round(math.sqrt(mse)))
    print("RMSLE",np.log(np.sqrt(mean_squared_error(y_test,y_pred))))
    print('R2 score:', r2)

In [17]:
print('There were {} columns before encoding categorical features'.format(dataset.shape[1]))
encoded_dataset = oneHotEncode(dataset, list(dataset.select_dtypes(include=['object']).columns))
print('There are {} columns after encoding categorical features'.format(encoded_dataset.shape[1]))

There were 51 columns before encoding categorical features
There are 79 columns after encoding categorical features


In [18]:
encoded_important_attributes = list(dataset[important_attributes].select_dtypes(exclude=['object']).columns)

for obj in list(dataset[important_attributes].select_dtypes(include=['object']).columns):
    prefix = obj + '_'
    encoded_important_attributes += [prefix + str(val) for val in dataset[obj].unique()]

In [19]:
# Split data into training and testing sets
X = encoded_dataset[encoded_important_attributes]
y = encoded_dataset['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

#### RandomForestRegressor

In [20]:
# Fit Random Forest Regression model
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

rf.score(X_train, y_train)

0.9688867229312317

In [21]:
y_pred = rf.predict(X_test)

In [22]:
get_results(y_test, y_pred)

Mean squared error: 40725557553
Root MSE: 201806
RMSLE 12.21506084101396
R2 score: 0.8020155975932953


#### GradientBoostingRegressor

In [23]:
gb = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train)

gb.score(X_train, y_train)

0.794184155252953

In [24]:
# Predict prices for test data
y_pred = gb.predict(X_test)

In [25]:
get_results(y_test, y_pred)

Mean squared error: 49745473785
Root MSE: 223037
RMSLE 12.3150926586064
R2 score: 0.758165916160759


#### XGBRegressor

In [26]:
XGBModel = XGBRegressor()
XGBModel.fit(X_train, y_train, verbose=False)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [27]:
# Get the mean absolute error on the validation data :
y_pred = XGBModel.predict(X_test)

In [28]:
get_results(y_test, y_pred)

Mean squared error: 47985973897
Root MSE: 219057
RMSLE 12.297087297339784
R2 score: 0.7667195997613458


#### Neural Network

In [29]:
model = Sequential()

# The Input Layer :
model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

2023-03-16 22:08:04.305582: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-16 22:08:04.311147: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-03-16 22:08:04.311163: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-03-16 22:08:04.311439: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neur

In [30]:
# Compile the network :
model.compile(loss='mean_squared_logarithmic_error', optimizer='adam', metrics=['mse', 'mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4480      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 169,345
Trainable params: 169,345
Non-trainable params: 0
_________________________________________________________________


In [31]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

In [32]:
model.fit(X_train, y_train, epochs=250, batch_size=32, validation_split = 0.2, verbose=0)

In [33]:
y_pred = model.predict(X_test)

31/31 [==============================] - 0s 858us/step


In [34]:
get_results(y_test, y_pred)

Mean squared error: 59676225318
Root MSE: 244287
RMSLE 12.406099771073105
R2 score: 0.7098882736718746
